In [85]:
import ee
import geemap
import os
import geopandas as gpd
import pandas as pd
import shutil

# Intermediate results can be shown on the map

In [86]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

# From a gee feature (point) to a Rectangle ROI (in UTM)

In [87]:
def create_roi(feature):
	buffer = feature.transform(ee.Projection(feature.get("EPSG")), ee.ErrorMargin(1, "meters")).buffer(330)
	#// Get bounding boxx and simplify it to approximate a North aligned square ROI
	return buffer.bounds().simplify(ee.ErrorMargin(1, "meters"))

def set_utm_epsg(feature):
	coords = feature.geometry().coordinates()
	lon = ee.Number(coords.get(0))
	lat = ee.Number(coords.get(1))
	epsg = ee.Number(32700).subtract(lat.add(45).divide(90).round().multiply(100)).add(lon.add(183).divide(6).round()).uint16()
	#print(epsg.getInfo())
	return feature.set("EPSG",ee.String("EPSG:").cat(ee.String(str(epsg.getInfo()))))#ee.String("EPSG:").cat

def add_xy(img):
	return img.addBands(ee.Image.pixelLonLat().rename(["X", "Y"]))

- **.getInfo()** (e.g., in Line 12 of the above cell) is alway necessary, the same when we want to print sth, which is not the case when using GEE directly

- get image patch for one gee feature (point)

# From a gee feature to a good s1/2 patch

In [88]:
def get_s1_mosaic(city):
    #input is a gee feature
    
    roi = city.geometry()

    def clip_roi(img):
        return img.clip(roi).unmask()

    collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate(timePeriod).filterBounds(roi).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select(s1_bands) \
    .map(clip_roi) \
    .median()
    return collection

def get_s2_cloud_free(city):
    #input is a gee feature
    
    roi = city.geometry()
    
    def clip_roi(img):
        return img.clip(roi).unmask()

    collection = ee.ImageCollection("COPERNICUS/S2") \
    .filterDate(timePeriod) \
    .filterBounds(roi) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', False) \
    .select(s2_bands) \
    .map(clip_roi) \
    .mosaic()
    return collection

# Download image patches

## Set path and para.

In [89]:
top_folder = r".\test\shapefile"

out_dir = r".\test\res"

timePeriod = ee.DateRange('2016-01-01', '2016-09-01')

s2_bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12']

s1_bands = ["VV", "VH"]

## Main steps:

- loop all the cities
- loop all the points in one city: convert each point to a gee feature
- find the image and download
- record number of points per class per city

In [90]:
id = 0
for path, dirs, files in os.walk(top_folder):


    #print(num_label)
    print('!!!!!!!!!!!!!!!!!', path, dirs, files)

    for d in dirs:
        sh_path = os.path.join(top_folder, d)
        print(d)

        file=os.path.join(sh_path, d+".shp")
        shapefile = gpd.read_file(file)
        print(shapefile.shape)

        num_label = pd.Series(0, index=range(1, 18))

        'prepare a empty city folder to save download images'
        cityFolder = os.path.join(out_dir, d)
        if os.path.exists(cityFolder):
            shutil.rmtree(cityFolder, ignore_errors=True)

        'download a patch for each point'
        for i in range(3):#shapefile.shape[0]

            'convert one row in the shapefile into a feature for gee'
            features = []

            geom = shapefile.iloc[i:i + 1, :]

            '''convert to gee feature'''
            jsonDict = eval(geom.to_json())
            geojsonDict = jsonDict['features'][0]
            feature=ee.Feature(geojsonDict)
            #print(feature.getInfo())

            #Get the individual geometries as a list.
            geometries = feature.geometry()
            #print(geometries.getInfo())

            pointsC = geometries.coordinates()
            print(pointsC.getInfo())

            label = feature.get("class").getInfo()

            #print(label, pointsC.size().getInfo())
            #print('Point 1', geometries.get(0))

            '''record the num of labels'''
            num_label[label]=num_label[label]+1

            'set path to save the download images'
            cityFolder_label = os.path.join(cityFolder, str(label))
            if not os.path.exists(cityFolder_label):
                    os.makedirs(cityFolder_label)
            filename = os.path.join(cityFolder_label, str(num_label[label])+'.tif')

            #print(feature.getInfo())
            feature_epsg = set_utm_epsg(feature)
            #print(feature_epsg)
            roiF = create_roi(feature_epsg)

            'find img'
            s2 = get_s2_cloud_free(roiF)
            s1 = get_s1_mosaic(roiF)
            
            'for debug'
            #print(s2.getInfo())
            Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000}, "S2")
            Map.addLayer(s1, {'bands': ['VV', 'VH', 'VV'], 'min': -30, 'max': 0}, "S1", False)
            #print(roiF.geometry().getInfo())
            Map.addLayer(roiF, {}, 'roiF')
            Map.centerObject(feature.geometry(), 15)
            Map

            'download'
            image = s2.addBands(s1)
            
            roi = ee.Geometry(roiF.geometry())
            #print(roi.getInfo())
            roi = roi.transform(feature_epsg.get("EPSG"), ee.ErrorMargin(1, "meters"))
            print(roi.getInfo())
            #the transformation here is to make sure that the downloaded images has the expected size (decided with buffer in function create_roi)
            
            geemap.ee_export_image(image, filename=filename, scale=10, region=roi, crs=ee.Projection(feature_epsg.get("EPSG")), file_per_band=False)#, region=roi

        if id == 0:
            label_per_City = pd.DataFrame(num_label,columns=[d])
        else:
            label_per_City[d] = num_label
        id = 1

        print(label_per_City)

!!!!!!!!!!!!!!!!! .\test\shapefile ['Berlin'] []
Berlin
(313, 3)
[13.384490951732916, 52.8168516904125]
{'geodesic': False, 'crs': {'type': 'name', 'properties': {'name': 'EPSG:32633'}}, 'type': 'Polygon', 'coordinates': [[[390792.3855393301, 5852797.305685352], [391450.30982119514, 5852782.522842562], [391465.10569716495, 5853442.998028512], [390807.2711287224, 5853457.780017628], [390792.3855393301, 5852797.305685352]]]}
Data downloaded to E:\0Learn\0LearningNotes_python_geo_rs\test\res\Berlin\11\1.tif
[13.853979321383182, 52.38866507060918]
{'geodesic': False, 'crs': {'type': 'name', 'properties': {'name': 'EPSG:32633'}}, 'type': 'Polygon', 'coordinates': [[[421675.8723533494, 5804561.037127061], [422333.81683314533, 5804550.611199544], [422344.2389448243, 5805211.075194277], [421686.38277303334, 5805221.500554834], [421675.8723533494, 5804561.037127061]]]}
Data downloaded to E:\0Learn\0LearningNotes_python_geo_rs\test\res\Berlin\11\2.tif
[13.12611866517181, 52.756230529405826]
{'ge

## Important notes:

- be careful of data CRS
- display/print interm results on map to avoid mistakes
- unsure code can be first tested with GEE: https://code.earthengine.google.com
- useful Ref: https://github.com/giswqs/earthengine-py-notebooks

# Question

- now the downloaded image patch is not of the same size, probably due to the buffer operation in function create_roi
- how to explicitly set the downloaded image patch?
- how to speed up, e.g., when there are many cities to be processed?